In [ ]:
print("ok")

ok


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).

In [ ]:
!pip install transformers sentence-transformers==2.2.2 langchain pinecone-client pypdf flask tiktoken python-dotenv langchain-community langchain-pinecone


In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
import torch
import transformers


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
model ="meta-llama/Llama-2-7b-chat-hf"

In [ ]:
tokenizer= AutoTokenizer.from_pretrained(model)

In [ ]:
pipeline= transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1500,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def loadPdf(data):
  loader =   DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)

  documents = loader.load()
  return documents

In [ ]:
extracted_data = loadPdf("data/")

In [ ]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap=50)
    text_chunk = text_splitter.split_documents(extracted_data)

    return text_chunk

In [ ]:
text_chunk = text_split(extracted_data)
print(len(text_chunk))

5830


In [ ]:
def download_hugging_embeddings():
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    print(embeddings)
    return embeddings

In [ ]:
embeddings = download_hugging_embeddings()

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} query_instruction='Represent this question for searching relevant passages: ' embed_instruction='' show_progress=False


In [ ]:
result = await embeddings.aembed_query("how are you")
print(result)


[0.024061311036348343, 0.14484883844852448, 0.06065581366419792, 0.024459004402160645, -0.017630474641919136, 0.02516491897404194, 0.048197731375694275, 0.01718091033399105, -0.03890782222151756, 0.002648566383868456, -0.006256014108657837, -0.0605681948363781, -0.01945561356842518, -0.012855424545705318, 0.026287013664841652, -0.051481783390045166, 0.03218235820531845, -0.061546046286821365, -0.0895797535777092, -0.0055295848287642, 0.04938984662294388, 0.0694296658039093, 0.043810490518808365, 0.00816299021244049, 0.010193023830652237, -0.010890694335103035, -0.0028137783519923687, -0.045275431126356125, 0.10420200973749161, -0.03859566152095795, -0.0626998096704483, -0.032719604671001434, -0.01710384339094162, 0.029902128502726555, -0.030825791880488396, 0.021428575739264488, 0.037119440734386444, -0.04666193574666977, 0.0711924359202385, -0.08809220045804977, 0.0074519747868180275, -0.0488310270011425, -0.005623843986541033, -0.038208987563848495, 0.07419692724943161, -0.0998404696

In [ ]:
import os
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY',"79cea564-8382-45be-91f1-01a3b003c16c")

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="79cea564-8382-45be-91f1-01a3b003c16c")

In [ ]:
index_name="lawchatbot"

In [ ]:
index=pc.Index("lawchatbot")

In [ ]:
vectorstore=PineconeVectorStore(embedding=embeddings, index_name=index_name,pinecone_api_key="79cea564-8382-45be-91f1-01a3b003c16c")

In [ ]:
vectorstore.add_texts([t.page_content for t in text_chunk])

['7226e07b-6367-4c18-b4f8-8def240f80ed',
 'b63e8ea8-71e7-46e5-8d71-a41fc59a7ba3',
 '4642da87-facf-43fa-9666-bf8ee70d7092',
 'd2df8360-0470-4bf9-bf6d-6492e6bca2fe',
 '184f60c0-97f1-41f5-90ea-1d72be97012c',
 'eef20fc1-a4a0-4159-8507-873b4bdc7971',
 '4ddf246d-e4a0-4d39-9b2c-6dc29c1fcc34',
 'edfc5778-b338-4d89-9b26-fccbd1ed22f1',
 '3ef758fc-b230-428b-9868-ecc9f7e2ab0d',
 'd8a8a514-a3d8-4282-ac7f-544fcfd422f4',
 'e882d8ab-879d-44f8-967a-9e1eb220b513',
 'a3561f00-7389-4143-ab20-774b40b60d9f',
 '3f63e83c-efb9-41b7-8416-7ba2059b59b5',
 'c2eff789-07af-4d0a-9c94-da5b3fb110bd',
 '02f64036-6a05-4d73-b8fa-a887a8b54156',
 '206ffc54-f397-4095-8293-5eaaff8d4a6f',
 'e835bfc7-b602-4096-8149-953ad3b5085a',
 '735023be-947f-467d-935c-cea00a4b2413',
 '068a62b4-5cb8-476c-a0c5-e8b2f52ada67',
 'ce64b64f-637c-42ca-b43b-e8299f5aa67a',
 '65d42d8d-7655-43b2-9630-c17f96263ad9',
 '2c406b1d-62a7-4ef0-acb6-80fb0b145b1e',
 '52dd9406-876f-427c-bf6c-fa8ca699cf75',
 '4471e154-437a-402e-9bf3-437fddcdafed',
 'a47045d2-8cbe-

In [ ]:
query = "Need for Cyber law"
docs = vectorstore.similarity_search(query, k=3)

# Print results
print("Result", docs)

Result [Document(page_content='regulatory mechanisms and legal infrastructures come within the domain of \nCyber law.  \nCyber law is important because it touches almost all aspects of transactions \nand activities on and involving the internet, World Wi de Web and cyberspace. \nEvery action and reaction in cyberspace has some legal and cyber legal \nperspectives.  \nCyber law encompasses laws relating to –  \n\uf0b7 Cyber crimes  \n\uf0b7 Electronic and digital signatures  \n\uf0b7 Intellectual property  \n\uf0b7 Data protection and privacy'), Document(page_content='3 \n 1. INTRODUCTION  \n"Cyber" is a prefix used to describe a person, thing, or idea as part of the \ncomputer and information age. Tak en from  kybernetes , Greek word for \n"steersman" or "governor," it was first used in  cybernetics, a word coined by \nNorbert Wiener and his colleagues.  The virtual world of internet is known as \ncyberspace and the laws governing this area are known as Cyber law s and all \nthe netize

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [1]:
llm=HuggingFacePipeline(pipeline=pipeline,model_kwargs={'temperature' : 0})

NameError: name 'HuggingFacePipeline' is not defined

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={'k': 4}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Input Prompt:Indian Penal Code, 1860


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Response :  
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: 170–72; marginalized  peoples and, 9, 17, 
42, 211, 219, 224, 227; nationalism and, 6, 
217; the  people and, 10, 25–27; postcolo -
nialism and, 7, 18, 216; prohibition and, 
33–34, 39–42, 49, 54, 219–20; property 
rights and, 30–31; prostitution and, 20, 
172–73, 177–78, 199–200, 202, 213, 219; 
public interest and, 18, 34;  women and,  
9, 17, 29–30, 171, 173–75
Indian Contagious Diseases Act, 183,  
185
Indian Council of Agricultural Research, 
159
Indian Council of Medical Research,  
159
Indian Councils Act (1909), 237n26
Indian Evidence Act (1872), 93
Indian High Courts Act (1861), 13–14
Indian In  de pen dence, 3, 215
Indian Law Institute, 117–18
Indian Merchants’ Chamber, 86, 97
Indian National Congress, 126
Indian Parliament, 87–88
Indian Penal Code, 32, 193
Indian Railways Act, 107–8
Indian Re

KeyboardInterrupt: Interrupted by user